# file set-up
1. import necessary libraries  
2. title directory (where data files are stored)  
3. create empty dataframe  
4. create list of languages
    will return to this---currently it doesn't actually serve a purpose, as I was unable to get the languages through here.  
    will either update this file or change spider to sort by language.  

In [2]:
from bs4 import BeautifulSoup as beau
import pandas as pd
import itertools
import pickle
import nltk
import os
import re

In [3]:
directory = 'natfinder/natfinder/scraped-works'
fics_df = pd.DataFrame(columns=['L1', 'author', 'title', 'tags', 'summary', 'notes', 'endnotes', 'work', 'rating', 'warnings', 'category', 'fandom', 'ships', 'charas', 'freeform', 'published', 'status', 'word_count', 'chaps', 'comments', 'kudos', 'hits', 'bookmarks'])
langs = ['french', 'spanish', 'mexican', 'brazilian', 'portugeuse', 'russian', 'ukranian', 'bengali', 'italian', 'czech', 'japanese', 'korean', 'chinese', 'swedish', 'german', 'finnish', 'turkish', 'greek', 'hindi']

# def: extract_info(soup)
1. try/except in case data is missing from file (ie; fics written by anonymous users will not return an author value)  
2. gather basic info (author, title, tags, summary, notes, endnotes, work)  
    tag info (rating, warning, category, fandom, (relation)ships, characters, freeform)  
    stat info (publication date, status, word count, chapters, comments, kudos, hits, bookmarks)
3. add to `fics_df`

In [ ]:
def extract_info(soup):
    # basic info
    try:
        author = soup.find(rel='author').text
    except:
        author = 'NaN'
    try:
        title = soup.find(class_='title heading').text
    except:
        title = 'NaN'
    try:
        tags = [t.text for t in soup.find_all(class_='tag')]
    except:
        tags = 'NaN'
    try:
        summary = soup.find('div', class_='summary module').text
    except:
        summary = 'NaN'
    try:
        notes = soup.find('div', class_='notes module').text
    except:
        notes = 'NaN'
    try:
        endnotes = soup.find('div', class_='end notes module').text
    except:
        endnotes = 'NaN'
    try:
        work = soup.text
    except:
        work = 'NaN'
    # tag info
    try:
        rating = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='rating tags')]
    except:
        rating = 'ERROR'
    try:
        warnings = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='warning tags')]
    except:
        warnings = 'ERROR'
    try:
        categories = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='category tags')]
    except:
        categories = 'ERROR'
    try:
        fandoms = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='fandom tags')]
    except:
        fandoms = 'ERROR'
    try:
        ships = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='relationship tags')]
    except:
        ships = 'NaN'
    try:
        charas = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='character tags')]
    except:
        charas = 'NaN'
    try:
        freeform = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='freeform tags')]
    except:
        freeform = 'NaN'
    # stats info
    try:
        published = soup.dd.find(class_='published').text
    except:
        published = 'ERROR'
    try:
        status = soup.dd.find(class_='status').text
    except:
        status = 'ERROR'
    try:
        wc = int(soup.dd.find(class_='words').text)
    except:
        wc = 'ERROR'
    try:
        chapters = int(soup.dd.find(class_='chapters').text)
    except:
        chapters = 'ERROR'
    try:
        comments = int(soup.dd.find(class_='comments').text)
    except:
        comments = 0
    try:
        kudos = int(soup.dd.find(class_='kudos').text)
    except:
        kudos = 0
    try:
        hits = int(soup.dd.find(class_='hits').text)
    except:
        hits = 0
    try:
        bookmarks = int(soup.dd.find(class_='bookmarks').text)
    except:
        bookmarks = 0
    temp = pd.DataFrame({'author':[author], 'title':[title],'tags':[tags], 'summary':[summary], 'notes':[notes], 'endnotes':[endnotes], 'work':[work], 'rating':[rating], 'warnings':[warnings], 'category':[categories], 'fandom':[fandoms], 'ships':[ships], 'charas':[charas], 'freeform':[freeform], 'published':[published], 'status':[status], 'word_count':[wc], 'chaps':[chapters], 'comments':[comments], 'kudos':[kudos], 'hits':[hits], 'bookmarks':[bookmarks]})
    return temp

# parsing through files
1. gather files  
2. create bs4 soup for files
3. use `extract_info(soup)` to add to fics_df

In [17]:
files = []
for f in os.scandir(directory):
    if f.name.endswith('.html'):
        files.append(f.name)
for f in files:
    soup = beau(open(directory + '/' + f).read(), 'html.parser')
    fics_df = pd.concat([fics_df, extract_info(soup)], ignore_index=True)

4. create additional data columns  
    may move this over to data_organization.ipynb
5. pickle dataframe as `fics_df.pkl`

In [11]:
with open('fics_df_scraped.pkl', 'wb') as file:
    pickle.dump(fics_df, file)

In [18]:
fics_df.describe

<bound method NDFrame.describe of        L1            author  \
0     NaN    orphan_account   
1     NaN       OffKeyPanda   
2     NaN          qamiqami   
3     NaN     beautifulpain   
4     NaN            RenSoo   
...   ...               ...   
1127  NaN   Piensodemasiado   
1128  NaN        AmericanPi   
1129  NaN  sixdollardropout   
1130  NaN           Ibbyliv   
1131  NaN    Lovegingernuts   

                                                  title  \
0                     \n      Bright Red Lanterns\n       
1                    \n      ¡Qué estas diciendo!\n       
2                          \n      Beautiful Mess\n       
3         \n      take only what you need from me\n       
4                            \n      Forgiveness.\n       
...                                                 ...   
1127                       \n      Baby Don't Cry\n       
1128                  \n      Before the Darkness\n       
1129              \n      A smutty Destiel ficlet\n       
113